In [ ]:
import os
import numpy as np
#import pandas as pd
os.chdir("../input/word2vec-nlp-tutorial")

In [ ]:
!unzip -o labeledTrainData.tsv.zip -d /kaggle/working
!unzip -o testData.tsv.zip -d /kaggle/working
!unzip -o unlabeledTrainData.tsv.zip -d /kaggle/working

In [ ]:
os.chdir('/kaggle/working')
with open('labeledTrainData.tsv','r',encoding='utf-8') as f1:
    trdata = f1.readlines()
    trlist = []
    trrate = []
    for sent in trdata[1:]:
        sen = sent.strip('\n').split('\t')
        trlist.append(sen[2].split(' '))
        trrate.append(sen[1])

In [ ]:
with open('testData.tsv','r',encoding='utf-8') as f1:
    tedata = f1.readlines()
    telist = []
    for sent in tedata[1:]:
        sen = sent.strip('\n').split('\t')
        telist.append(sen[1].split(' '))
with open('unlabeledTrainData.tsv','r',encoding='utf-8') as f1:
    tr_nolabel = f1.readlines()
    tr_nolist = []
    for sent in tr_nolabel[1:]:
        sen = sent.strip('\n').split('\t')
        tr_nolist.append(sen[1].split(' '))

In [ ]:
import argparse
from gensim.models import word2vec
from time import time

path_prefix = './'
worddata = trlist+telist+tr_nolist
print("the len of data is {}".format(len(worddata)))
st=time()
model = word2vec.Word2Vec(worddata,vector_size=250, window=5, min_count=5, workers=12, epochs=10, sg=0)
model.save(os.path.join(path_prefix, 'w2v_all.model'))
print("we use time {} min".format((time()-st)/60))

In [ ]:
model.

In [ ]:
word2idx = model.wv.key_to_index
idx2word = model.wv.index_to_key
embeddingmatrix = []
for i,v in enumerate(idx2word):
    embeddingmatrix.append(model.wv[v])
#"PAD" vector
vec1 = np.random.rand(250)
#"UNK" vector
vec2 = np.random.rand(250)
word2idx["PAD"] = len(word2idx)
idx2word.append("PAD")
embeddingmatrix.append(vec1)
word2idx["UNK"] = len(word2idx)
idx2word.append("UNK")
embeddingmatrix.append(vec2)
#embeddingmatrix = torch.cat([embeddingmatrix,vec1,vec2],0)

In [ ]:
# set the length of word is 40
senlen = 100
train_num = []
for senten in trlist:
    wordlist = []
    if len(senten) >= senlen:
        for sen in senten[:senlen]:
            if sen in word2idx.keys():
                wordlist.append(word2idx[sen])
            else:
                wordlist.append(word2idx['UNK'])
        train_num.append(wordlist)
    else:
        less_num = senlen - len(senten)
        for sen in senten:
            if sen in word2idx.keys():
                wordlist.append(word2idx[sen])
            else:
                wordlist.append(word2idx['UNK'])
        for _ in range(less_num):
            wordlist.append(word2idx['PAD'])
        train_num.append(wordlist)

In [ ]:
import torch
from torch.utils import data
# define tordata type
class moviedata(data.Dataset):
    def __init__(self, X, y):
        self.data = X
        self.label = y
    def __getitem__(self, idx):
        if self.label is None: return self.data[idx]
        return self.data[idx], self.label[idx]
    def __len__(self):
        return len(self.data)

In [ ]:
from torch import nn
class gru_net(nn.Module):
    def __init__(self,embedding, hidden_dim, num_layers, dropout=0.5,embedding_train=False):
        super(gru_net, self).__init__()
        self.embedding = nn.Embedding(embedding.size(0),embedding.size(1))
        self.embedding.weight = nn.Parameter(embedding)
        self.embedding.weight.requires_grad = embedding_train
        self.embedding_dim = embedding.size(1)
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.dropout = dropout
        self.gru = nn.GRU(self.embedding_dim,self.hidden_dim,num_layers=self.num_layers,batch_first=True)
        self.classfier = nn.Sequential(
            nn.Dropout(self.dropout),
            nn.Linear(self.hidden_dim,1),
            nn.Sigmoid()
        )
    def forward(self,inputs):
        inputs = self.embedding(inputs)
        inputs = inputs.float().cuda()
        x, _ = self.gru(inputs, None)
        x = x[:,-1,:]# x is [batch,seq_len,hidden_size]
        x = self.classfier(x)
        return x

In [ ]:
def evaluation(outputs, labels):
    # outputs => probability (float)
    # labels => labels
    outputs[outputs>=0.5] = 1 # 大於等於 0.5 為有惡意
    outputs[outputs<0.5] = 0 # 小於 0.5 為無惡意
    correct = torch.sum(torch.eq(outputs, labels)).item()
    return correct

In [ ]:
import os
import torch
import argparse
import numpy as np
from torch import nn
from gensim.models import word2vec
#from sklearn.model_selection import train_test_split

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
embedding_train=False # fix embedding during training
batch_size = 128
epoch = 10
lr = 0.001
model_dir = './'

print(len(train_num))
X = torch.LongTensor(train_num)
y = [int(rate) for rate in trrate]
y = torch.LongTensor(y)
embeddingmatrix = torch.from_numpy(np.array(embeddingmatrix))
#embeddingmatrix = torch.tensor(embeddingmatrix)
#establish model
gmodel = gru_net(embeddingmatrix, hidden_dim=150, num_layers=1, dropout=0.5,embedding_train=embedding_train)
gmodel = gmodel.to(device)
# split data
x_train,x_valid,y_train,y_valid = X[:18000],X[18000:],y[:18000],y[18000:]
# dataloader
train_dataset = moviedata(X=x_train, y=y_train)
val_dataset = moviedata(X=x_valid, y=y_valid)

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                            batch_size = batch_size,
                                            shuffle = True,
                                            num_workers = 4)

val_loader = torch.utils.data.DataLoader(dataset = val_dataset,
                                            batch_size = batch_size,
                                            shuffle = False,
                                            num_workers = 4)
torch.cuda.empty_cache()
train_process(batch_size, epoch, lr, model_dir, train_loader, val_loader, gmodel, device)